In [1]:
%pip install keras
%pip install tensorflow
%pip install numpy
%pip install pandas

You should consider upgrading via the '/home/justyn/datamining/dm-finalproj/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/justyn/datamining/dm-finalproj/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/justyn/datamining/dm-finalproj/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/justyn/datamining/dm-finalproj/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM

2023-03-30 18:22:32.551433: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-30 18:22:32.595118: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-30 18:22:32.596390: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 18:22:33.301033: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Load in the dataset
df1 = pd.read_csv("../resources/sentiment_scores.csv")
df2 = pd.read_csv("../resources/raw-dataset.csv")
df = pd.merge(df1, df2, on='row_id')
df.head()

,Unnamed: 0,row_id,text_x,pos,neg,neu,compound,category,rating,label,text_y
0,0,1,love this well made sturdy and very comfor...,0.715,0.000,0.285,0.9558,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,1,2,love it a great upgrade from the original i...,0.491,0.000,0.509,0.8910,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,2,3,this pillow saved my back i love the look and...,0.389,0.000,0.611,0.7906,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,3,4,missing information on how to use it but it i...,0.279,0.079,0.642,0.7227,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,4,5,very nice set good quality we have had the s...,0.281,0.000,0.719,0.7397,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [4]:
# Create a new column called real, if label==CG set to 0, or False, if set to OR set to 1, or True
df['real'] = df['label'].apply(lambda x: 0 if x == 'CG' else 1)
# Randomize the order of the data
df = df.sample(frac=1).reset_index(drop=True)
# Split into train and test data
train = df[:int(len(df)*0.8)]
test = df[int(len(df)*0.8):]

In [12]:
# For training we only need the scores and the label
train_x = train[['pos', 'neg', 'neu', 'compound']]
train_y = train['real']

test_x = test[['pos', 'neg', 'neu', 'compound']]
test_y = test['real']

In [13]:
# Reshape the data to be 3D
train_x = np.array(train_x).reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = np.array(test_x).reshape((test_x.shape[0], 1, test_x.shape[1]))

# change the train data to be a float type
train_x = train_x.astype('float32')
test_x = test_x.astype('float32')

In [15]:
# Create the model
model = Sequential()
model.add(LSTM(100, input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-03-30 18:24:23.018192: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-30 18:24:23.140219: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-30 18:24:23.141642: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [16]:
# Train
history = model.fit(train_x, train_y, epochs=500, batch_size=16, validation_data=(test_x, test_y), verbose=2, shuffle=False)

Epoch 1/500


2023-03-30 18:24:25.159545: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-30 18:24:25.161324: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-30 18:24:25.162590: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2022/2022 - 9s - loss: 0.6833 - accuracy: 0.5708 - val_loss: 0.6788 - val_accuracy: 0.5974 - 9s/epoch - 4ms/step
Epoch 2/500
2022/2022 - 7s - loss: 0.6776 - accuracy: 0.5826 - val_loss: 0.6768 - val_accuracy: 0.5929 - 7s/epoch - 3ms/step
Epoch 3/500


KeyboardInterrupt: 

In [9]:
# These will be our x data